# CSS 120

## Climate Change Models

### Umberto Mignozzetti (UCSD)

(Based on Project Pythia and ClimateMatch)

## Today's lecture

In this lecture, we will study the physics of:

1. Radiation and greenhouse effect
1. Energy Balance
1. Zero-Dimensional Energy Balance Model
1. Climate Change Scenarios
1. Climate feedbacks

## Packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import holoviews as hv
import panel as pn
from scipy.optimize import brentq

# Radiation and the Greenhouse Effect

## Planck's Law

All objects with a temperature emit **[electromagnetic radiation](https://www.noaa.gov/jetstream/satellites/electromagnetic-waves)**. 

This energy travels through space in the form of waves. 

We will use the blackbody model as a basis to understand Earth's energy balance. 

> In climate science, the blackbody model refers to the concept of a “blackbody” in physics, which is an idealized object that absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence, and re-radiates energy perfectly according to its temperature.

## Planck's Law

If we suppose Earth behaves as a perfect blackbody, then it emits energy at all wavelengths according to **[Planck's Law](https://glossary.ametsoc.org/wiki/Planck%27s_radiation_law)**:

\begin{align}
B(\lambda,T) = \frac{2 h c^2}{\lambda^5}\frac{1}{e^{hc/(\kappa T)}-1}
\end{align}

Where $h =  6.626075 \times 10^{-34} \text{ J s}$ is Planck's constant, $c= 2.99792 \times 10^8 \text{ m s}^{-1}$ is the speed of light, and $\kappa = 1.3804 \times 10^{23} \text{ W K}^{-1}$ is Boltzmann's constant.

## Planck's Law

Let us simulate how the blackbody curve changes as Earth warms and cools relative to its current surface temperature of about 288 K. Use the slide bar to adjust the emission temperature.

Give the code a few seconds to replot before choosing a new temperature.

No need to worry about understanding the code here - this is conceptual.

Make sure you execute this cell to enable the widget! Please run this cell **twice** to be able to use the slides bar.

## Planck's Law

In [ ]:
# constants
h = 6.626075e-34  # J s
c = 2.99792e8  # m s^-1
k = 1.3804e-23  # W K^-1

# Planck's Law depending on wavelength (lambda) and temperature (T)
def planck(wavelength, temperature):
    a = 2.0 * h * c**2
    b = h * c / (wavelength * k * temperature)
    intensity = a / ((wavelength**5) * (np.exp(b) - 1.0))

    lpeak = (2.898 * 1e-3) / temperature

    return intensity

## Planck's Law

In [ ]:
def update_plot(emiss_temp):
    # generate x-axis in increments from 1um to 100 micrometer in 1 nm increments
    # starting at 1 nm to avoid wav = 0, which would result in division by zero.
    wavelengths = np.arange(1e-6, 50e-6, 1e-9)

    # get the blackbody curve and peak emission wavelength for 288 K
    intensity288 = planck(wavelengths, 288)

    # get the blackbody curve and peak emission wavelength for selected temperature
    intensity = planck(wavelengths, emiss_temp)

    #     # get the intensity at peak wavelength to limit the lines
    #     Ipeak,_ = planck(lpeak,emission_temperature)
    #     Ipeak288,_ = planck(lpeak288,288)

    # curves output
    vary = zip(wavelengths * 1e6, intensity)
    init_val = zip(wavelengths * 1e6, intensity288)

    # Specified individually
    list_of_curves = [
        hv.Curve(init_val, label="T=288K").opts(ylim=(0, 1.0e7)),
        hv.Curve(vary, label="T=" + str(emiss_temp) + "K").opts(ylim=(0, 1.0e7)),
    ]

    bb_plot = hv.Overlay(list_of_curves).opts(
        height=300,
        width=600,
        xlabel="Wavelength (μm)",
        ylabel="B(λ,T) (W/(m³ steradian)",
        title="Spectral Radiance",
    )

    return bb_plot

## Planck's Law

In [ ]:
hv.extension("bokeh")

emiss_temp_widget = pn.widgets.IntSlider(
    name="Emission Temperature", value=288, start=250, end=300
)
bound_plot = pn.bind(update_plot, emiss_temp=emiss_temp_widget)

pn.Row(emiss_temp_widget, bound_plot)

## Climate Connection

The **electromagnetic spectrum** (shown below), which displays the different wavelengths of electromagnetic energy.

![Diagram of the Electromagnetic Spectrum](https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/EM_spectrumrevised.png/1920px-EM_spectrumrevised.png)
Diagram of the Electromagnetic Spectrum. (Credit: [Wikipedia](https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/EM_spectrumrevised.png/1920px-EM_spectrumrevised.png))

According to our model and noting that 1 micrometer = $10^{-6}$ meters, with a surface temperature of 288 K what type of radiation does Earth primarily emit at? 

## Stefan-Boltzmann Law

If we integrate Planck's Law over all wavelengths and outward angles of emission, the total **outgoing longwave radiation (OLR)** for a given **emission temperature** ($\mathbf{T}$) follows the **[Stefan-Boltzmann Law](https://glossary.ametsoc.org/wiki/Stefan-boltzmann_law)**.

\begin{align}
OLR = \sigma T^4
\end{align}

Where the **[Stefan-Boltzmann constant](https://glossary.ametsoc.org/wiki/Stefan-boltzmann_constant)** is $\sigma = 5.67 \times 10^{-8}\text{ W m}^{-2} \text{ K}^{-4}$.

## Stefan-Boltzmann Law

Rearranging the equation above, we can solve for the emission temperature of Earth, $T$.

\begin{align}
T = \sqrt[4]{\frac{OLR}{\sigma}}
\end{align}

Using $OLR = 239\text{ W m}^{-2}$ we can calculate the value of $T$:

## Stefan-Boltzmann Law

In [ ]:
# define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
sigma = 5.67e-8  # W m^-2 K^-4

# define the outgoing longwave radiation based on observations from the IPCC AR6 Figure 7.2
OLR = 239  # W m^-2

# plug into equation
T = (OLR / sigma) ** (1 / 4)

# display answer
print("Emission Temperature: ", T, "K or", T - 273, "°C")

## Stefan-Boltzmann Law (Your turn)

1. How does this compare to the actual global mean surface temperature of $~288 \text{ K} / 15\text{°C}$?
2. Using $T = 288 \text{ K}$ would you expect the corresponding outgoing longwave radiation to be higher or lower than the observed $239 \text{ W m}^{-2}$?
3. What could be accounted for in this model to make it more realistic?

## Stefan-Boltzmann Law (Your turn)

By modifying the code above and solving for OLR, find the outgoing longwave radiation expected for the observed surface temperature of $288 \text{ K}$.

In [ ]:
# define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
sigma = ...

# define the global mean surface temperature based on observations
T = ...

# plug into equation
OLR = ...

# display answer
print("OLR: ", OLR, "W m^2")

# The Greenhouse Effect

## The Greenhouse Effect

The expected surface temperature using the blackbody radiation model is much colder than we observe it to be.

In this model, we assumed there is nothing that lies between Earth's surface and space that interacts with Earth's emitted radiation.

From the initial lecture on the global energy budget we know this is not true.

Earth has an atmosphere, and within it are many gases that interact with radiation in the infrared range at which Earth primarily emits.

> The effect of these gases on radiation, called the **[greenhouse effect](https://glossary.ametsoc.org/wiki/Greenhouse_effect#:~:text=As%20used%20in%20the%20field,absorb%20and%20emit%20infrared%20radiation.)**, is what warms earth to a habitable temperature. 

## The Greenhouse Effect

The gases that are responsible for this ([carbon dioxide](https://glossary.ametsoc.org/wiki/Carbon_dioxide), [water vapor](https://glossary.ametsoc.org/wiki/Water_vapor), [methane](https://glossary.ametsoc.org/wiki/Methane), [ozone](https://glossary.ametsoc.org/wiki/Ozone), [nitrous oxide](https://glossary.ametsoc.org/wiki/Nitrous_oxide), and [chloroflourocarbons](https://glossary.ametsoc.org/wiki/Chlorofluorocarbons)) are termed **[greenhouse gases](https://glossary.ametsoc.org/wiki/Greenhouse_gases)**, and are all **[trace gases](https://glossarytest.ametsoc.net/wiki/Trace_gas)** present in very low concentrations in Earth's atmosphere. **[Anthropogenic](https://glossarytest.ametsoc.net/wiki/Anthropogenic)** climate change is caused to a large extent by increased concentrations of many of these trace gases due to emissions from human activities.
 
The figure below shows the contributions to the global surface air temperature change relative to 1750.

We can see that all of the greenhouse gases have contributed positively, that is towards warming Earth.

Also, note that the total curve tracks the volcano curve quite well until around the 1850s when industrialization took hold.

The total and volcanic curves begin to deviate here, and after the mid-1900s the total curve begins tracking the total anthropogenic curve instead.

## The Greenhouse Effect

![Attributed global surface air temperature change (GSAT)](https://www.ipcc.ch/report/ar6/wg1/downloads/figures/IPCC_AR6_WGI_Figure_7_8.png)  
Figure 7.8 | Attributed global surface air temperature change (GSAT) from 1750 to 2019 produced using the two-layer emulator (Supplementary Material 7.SM.2), forced with ERF derived in this chapter (displayed in Figure 2.10) and climate response constrained to assessed ranges for key climate metrics described in Cross-Chapter Box 7.1. The results shown are the medians from a 2237-member ensemble that encompasses uncertainty in forcing and climate response (year-2019 best estimates and uncertainties are shown in Figure 7.7 for several components). Temperature contributions are expressed for carbon dioxide (CO2), methane (CH4), nitrous oxide (N2O), other well-mixed greenhouse gases (WMGHGs), ozone (O3), aerosols, and other anthropogenic forcings, as well as total anthropogenic, solar, volcanic, and total forcing. Shaded uncertainty bands show very likely (5–95%) ranges. Further details on data sources and processing are available in the chapter data table (Table 7.SM.14). (Credit [IPCC](https://www.ipcc.ch/report/ar6/wg1/downloads/figures/IPCC_AR6_WGI_Figure_7_8.png))

## A simple greenhouse model

*This section draws content from [The Climate Laboratory](https://brian-rose.github.io/ClimateLaboratoryBook/) by Brian E. J. Rose.*

As shown above, greenhouse gases are incredibly important for regulating Earth's energy balance and temperature.

A first approach is to model the greenhouse effect on outgoing longwave radiation (OLR) to space by adding a transmissivity coefficient. 

The **transmissivity coeficient (**$\mathbf{\tau}$**)** is the fraction of 
the radiation emitted from Earth that actually makes it to space. 

## A simple greenhouse model

This coefficient $\mathbf{\tau}$ is a number that lies between 0 and 1, and represents the *effects* of all the greenhouse gases on radiation, rather than including them explicity in the model.

This approach is called a **[parametrization](https://glossary.ametsoc.org/wiki/Parameterization)**.

Applying this to the original model for blackbody radiation, the modified model is

\begin{align}
OLR = \tau \sigma T^4
\end{align}

Using  $OLR = 239 \text{ W m}^{-2}$ and $T = 288 \text{ K}$, we can estimate $\tau$.

## A simple greenhouse model

In [ ]:
# define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
sigma = 5.67e-8  # W m^-2 K^-4

# define the outgoing longwave radiation based on observations from the IPCC AR6 Figure 7.2
OLR = 239  # W m^-2

# define the emission temperature based on observtions of global mean surface temperature
T = 288  # K

# plug into equation
tau = OLR / (sigma * T**4)  # unitless number between 0 and 1

# display answer
print("Transmissivity Coefficient: ", tau)

## A simple greenhouse model (Your turn)

1.  For a generic planet, what could be said about the planet's atmosphere when $\tau$ is close to 1? Close to 0? Use the OLR seen at the top of the atmosphere in your answer.

2.  In terms of energy received from the sun, what does only modifying the _OLR_ to account for the greenhouse effect imply? Are there any greenhouse gases you think would make this implication problematic?

3.  Is there any other part of the atmosphere aside from greenhouse gases that we have not discussed that would also affect $\tau$?

## A simple greenhouse model (Your turn again)

Using list comprehension, calculate the OLR for three values of $\tau = 0.2,0.6114,0.8$. 

Then plot this on a bar chat to compare. This should help you answer one of the questions above.

Hint: what is [list comprehension](https://foundations.projectpythia.org/foundations/quickstart.html#lists)?

In [ ]:
# define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
sigma = ...

# define the emission temperature based on observtions of global mean surface temperature
T = ...

# define values of tau
tau = ...

# get values of OLR from tau using list comprehension
OLR = ...

# convert tau to list of strings using list comprehension so we can create a categorical plot
tau = ...

fig, ax = plt.subplots()
_ = ...
ax.set_xlabel("Transmissivity")
ax.set_ylabel("Outgoing Longwave Radiation (W m$^{-2}$)")

# Energy Balance

## Incoming Solar Radiation (Insolation) and Albedo ($\alpha$)

Just as Earth emits radiation, so does the sun. 

Radiation from the sun is referred to as *solar* or **[shortwave radiation](https://glossarytest.ametsoc.net/wiki/Shortwave_radiation)**. 

We will denote the incoming solar radiation, or **[insolation](https://glossary.ametsoc.org/wiki/Insolation)**, as $Q$. 

From the 'All Sky' Energy budget shown below, the insolation is observed to be $Q = 340 \text{ W m}^{-2}$. 

## Incoming Solar Radiation (Insolation) and Albedo ($\alpha$)

_Note that 'All Sky' refers to fluxes that **include the effects of clouds**. Compare the 'All Sky' figure to the 'Clear Sky' figure below, to see how the fluxes differ when measured only over regions that are cloud-free._

Some of this radiation is reflected back to space (for example off of ice and snow or clouds). We will denote the reflected flux back to space as $F_{\text{ref}}$. 

From the 'All Sky' energy budget below, this reflected flux is $F_{\text{ref}} = 100 \text{ W m}^{-2}$.

## Incoming Solar Radiation (Insolation) and Albedo ($\alpha$)

![Global Mean Energy Budget](https://www.ipcc.ch/report/ar6/wg1/downloads/figures/IPCC_AR6_WGI_Figure_7_2.png)
Figure 7.2 | Schematic representation of the global mean energy budget of the Earth (upper panel), and its equivalent without considerations of cloud effects (lower panel). Numbers indicate best estimates for the magnitudes of the globally averaged energy balance components in $\text{ W m}^{-2}$ together with their uncertainty ranges in parentheses (5–95% confidence range), representing climate conditions at the beginning of the 21st century. Note that the cloud-free energy budget shown in the lower panel is not the one that Earth would achieve in equilibrium when no clouds could form. It rather represents the global mean fluxes as determined solely by removing the clouds but otherwise retaining the entire atmospheric structure. This enables the quantification of the effects of clouds on the Earth energy budget and corresponds to the way clear-sky fluxes are calculated in climate models. Thus, the cloud-free energy budget is not closed and therefore the sensible and latent heat fluxes are not quantified in the lower panel. Figure adapted from Wild et al. (2015, 2019). (Credit: [IPCC AR6 Report](https://www.ipcc.ch/report/ar6/wg1/downloads/figures/IPCC_AR6_WGI_Figure_7_2.png))

## Incoming Solar Radiation (Insolation) and Albedo ($\alpha$)

The _fraction_ of reflected radiation relative to the insolation is called the **[planetary albedo](https://glossarytest.ametsoc.net/wiki/Planetary_albedo)** or just **albedo (**$\mathbf{\alpha}$**)**

\begin{align}
\alpha = \frac{F_{\text{ref}}}{Q}
\end{align}

Albedo is a unitless number between 0 and 1. We can use this formula to find the albedo of Earth.

## Incoming Solar Radiation (Insolation) and Albedo ($\alpha$)

In [ ]:
# define the observed insolation based on observations from the IPCC AR6 Figure 7.2
Q = 340  # W m^-2

# define the observed reflected radiation based on observations from the IPCC AR6 Figure 7.2
F_ref = 100  # W m^-2

# plug into equation
alpha = F_ref / Q  # unitless number between 0 and 1

# display answer
print("Albedo: ", alpha)

## Absorbed Shortwave Radiation (ASR)

The **absorbed shortwave radiation (ASR)** is the amount of insolation that is _not_ reflected, and actually makes it to Earth's surface. Thus,

\begin{align}
ASR = Q-F_{\text{ref}} = (1-\alpha)Q
\end{align}

From observations, we can estimate the absorbed shortwave radiation.

## Absorbed Shortwave Radiation (ASR)

In [ ]:
# plug into equation
ASR = (1 - alpha) * Q

# display answer
print("Absorbed Shortwave Radiation: ", ASR, " W m^-2")

## Equilibrium Temperature

Energy Balance is achieved when radiation absorbed by Earth's surface (ASR) is equal to longwave radiation going out to space (OLR). That is

\begin{align}
ASR = OLR
\end{align}

By substituting into the equations from previous sections, we can find the surface temperature of Earth needed to maintain this balance. This is called the **equilibrium temperature (** $\mathbf{T_{\text{eq}}}$ **)**.

Recall $OLR = \tau\sigma T^4$ and $ASR = (1-\alpha)Q$. The **equilibrium temperature** is the temperature the system would have if the energy balance was perfectly reached. 

## Equilibrium Temperature

Assuming energy balance, we will call the emission temperature denoted previously the equilibrium temperature ($T_{\text{eq}}$) instead. Thus,

\begin{align}
(1-\alpha)Q = ASR = OLR = \tau\sigma T_{\text{eq}}^4
\end{align}

Solving for $T_{\text{eq}}$ we find

\begin{align}
T_{\text{eq}} = \sqrt[4]{\frac{(1-\alpha)Q}{\tau\sigma}}
\end{align}

Let's calculate what this should be for Earth using observations:

## Equilibrium Temperature

In [ ]:
# define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
sigma = 5.67e-8  # W m^-2 K^-4

tau = 0.6127  # unitless number between 0 and 1

# plug into equation
T_eq = (((1 - alpha) * Q) / (tau * sigma)) ** (1 / 4)

# display answer
print("Equilibrium Temperature: ", T_eq, "K or", T_eq - 273, "°C")

## Increasing Greenhouse Gas Concentrations

Assume due to the increasing presence of greenhouse gases in the the atmosphere, that $\tau$ decreases to $0.57$.

We can then use our climate model and python to find the new equilibrium temperature.

In [ ]:
# define transmissivity (assupmtion in this case)
tau_2 = 0.57  # unitless number between 0 and 1

# plug into equation
T_eq_2 = (((1 - alpha) * Q) / (tau_2 * sigma)) ** (1 / 4)

# display answer
print("New Equilibrium Temperature: ", T_eq_2, "K or", T_eq_2 - 273, "°C")

# Zero-Dimensional Energy Balance Model

## Moving Forward in Time

One of the crucial things missing from the simple model we have looked at so far is its ability to _change with time_. 

As the composition of the atmosphere (among other things) changes, in response, so does the energy balance and global mean surface temperature. 

This is shown by the thick black lines in the figure below, where the time series of observed global mean surface air temperature change from the 1850-1900 reference period is plotted. 

Figures like this are called 'hockey stick' figures due to their shape: a relatively stable period followed by a steep increase

## Moving Forward in Time

![Observerd and simulated change in temperature](https://www.ipcc.ch/report/ar6/wg1/downloads/figures/IPCC_AR6_WGI_Figure_3_4.png)
Figure 3.4 | Observed and simulated time series of the anomalies in annual and global mean surface air temperature (GSAT). All anomalies are differences from the 1850–1900 time mean of each individual time series. The reference period 1850–1900 is indicated by grey shading. (a) Single simulations from CMIP6 models (thin lines) and the multi-model mean (thick red line). Observational data (thick black lines) are from the Met Office Hadley Centre/Climatic Research Unit dataset (HadCRUT5), and are blended surface temperature (2 m air temperature over land and sea surface temperature over the ocean). All models have been subsampled using the HadCRUT5 observational data mask. Vertical lines indicate large historical volcanic eruptions. CMIP6 models which are marked with an asterisk are either tuned to reproduce observed warming directly, or indirectly by tuning equilibrium climate sensitivity. Inset: GSAT for each model over the reference period, not masked to any observations. (b) Multi-model means of CMIP5 (blue line) and CMIP6 (red line) ensembles and associated 5th to 95th percentile ranges (shaded regions). Observational data are HadCRUT5, Berkeley Earth, National Oceanic and Atmospheric Administration NOAAGlobalTemp-Interim and Kadow et al. (2020). Masking was done as in (a). CMIP6 historical simulations were extended with SSP2-4.5 simulations for the period 2015–2020 and CMIP5 simulations were extended with RCP4.5 simulations for the period 2006–2020. All available ensemble members were used (see Section 3.2). The multi-model means and percentiles were calculated solely from simulations available for the whole time span (1850–2020). The figure is updated from Bock et al. (2020), their Figures 1 and 2. CC BY 4.0 https://creativecommons.org/licenses/by/4.0/. Further details on data sources and processing are available in the chapter data table (Table 3.SM.1). (Credit: [IPCC Report AR6](https://www.ipcc.ch/report/ar6/wg1/downloads/figures/IPCC_AR6_WGI_Figure_3_4.png))


## Moving Forward in Time

In order to incorporate time-dependent behavior into our model, we need to include a mathematical representation of how the _change in energy_ relates to a _change in temperature_ over time.

We can represent the change in temperature over time as the net heat input or loss from radiation ($ASR-OLR$) multiplied by the heat capacity of the Earth system.

As we learned, the heat capacity of a medium is its ability to increase in temperature given an input of heat. Not all components of the Earth system (for example land, ocean, atmosphere) have the same heat capacity.

## Moving Forward in Time

Mathematically, the rate of change of global mean surface temperature ($T$) over time is given as

\begin{align*}
\text{rate of change of }T &= \frac{1}{\text{heat capacity}}\cdot (\text{energy flux in - energy flux out}) \\
&= \frac{1}{C} \cdot ASR - OLR
\end{align*}

where $C$ is the heat capacity of the Earth system. Note here that when $ASR > OLR$, then the system is gaining heat. Conversely when $ASR < OLR$, then the system is losing heat over time.

To calculate the heat capacity for the Earth system, we will assume that it is a combination of atmosphere and ocean only, that is $C = C_{\text{oc}} + C_{\text{atmo}}$.

## Moving Forward in Time

Generally, the heat capacity of a medium is the specific heat of that medium times the total mass.

For the atmosphere, we have $C_{\text{atm}} = c_{\text{atm}} \cdot \frac{W_{\text{atm}}}{g} $ where $c_{\text{atm}}$ is the specific heat of the atmosphere, $W_{\text{atm}}$ is the weight of a column of air, and $g$ is the acceleration due to gravity.

For the ocean we have $C_{\text{oc}} = c_{\text{oc}} \cdot \rho_{\text{oc}} \cdot d_{\text{oc}}$ where $c_{\text{oc}}$ is the specific heat of the ocean, $\rho_{\text{oc}}$ is the density of seawater, and $d_{\text{oc}}$ is a representative depth of the ocean.

## Moving Forward in Time

In [ ]:
# heat capacity of the ocean
c_oc = 3850     # specific heat of seawater in J/kg/K
rho_oc = 1025   # average density of seawater in kg/m3
d_oc = 70       # depth of water in m (here representative of the mixed layer)
C_oc = c_oc * rho_oc * d_oc  # heat capacity of the ocean

# heat capacity of the atmosphere
c_atm = 1004    # specific heat of the atmosphere at constant pressure in J/kg/K
W_atm = 100000  # weight (pressure) of atmospheric column in Pa
g = 9.81        # acceleration due to gravity in m/s^2
C_atm = c_atm * (W_atm / g)  # heat capacity of the atmosphere

# total heat capacity
C = C_oc + C_atm

# print results
print(f'Ocean Heat Capacity:      {C_oc:.2f} J m^-2 K^-1')
print(f'Atmosphere Heat Capacity:  {C_atm:.2f} J m^-2 K^-1')
print(f'Total Heat Capacity:      {C:.2f} J m^-2 K^-1')

## Numerical Model

Knowing the heat capacity, and the descriptions of $OLR$ and $ASR$ from previous section, we can write the equation

First note that 
\begin{align}
\text{rate of change }T = \frac{\text{change in }T}{\text{change in time}}=\frac{dT}{dt}
\end{align}

So the equation for our model can be written
\begin{align}
\frac{dT}{dt}= \frac{1}{C}(ASR - OLR)
\end{align}

Numerically, we can use this equation to compute the global mean surface temperature after a small interval of time by adding on the amount of energy gained or lost multiplied by the time interval itself.

## Numerical Model

The particular method of numerically defining the time and temperature intervals (changes) is called **[discretization](https://glossarytest.ametsoc.net/wiki/Discretization)**, and the way we have chosen to do this is called the **Forward Euler method**.

The exact details of this method are beyond the scope of this class, and we will use the method without further elaboration.

The Forward Euler method assumes we can use $\text{change in }T = T_{n+1} - T_{n}$ and $\text{change in t} = t_{n+1} - t_{n}$ where $t$ is time.

Thus, if we know the time interval and the current temperature $\left(T_n\right)$, we can predict the temperature at the end of our time interval, $\left(T_{n+1}\right)$.

## Numerical Model

In [ ]:
# define the time interval, currently one year expressed in seconds
dt = 60.0 * 60.0 * 24.0 * 365.0

# define albedo
alpha = 0.2941

# define transmissivity
tau = 0.6127    # unitless number between 0 and 1


# define a function for absorbed shortwave radiation (ASR)
def ASR(alpha, Q):
    return (1 - alpha) * Q

## Numerical Model

In [ ]:
# define a function for outgoing longwave radiation (OLR)
def OLR(tau, T):
    # define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
    sigma = 5.67e-8  # W m^-2 K^-4

    return tau * sigma * T**4

# create a function to find the new temperature based on the previous using Euler's method.
def step_forward(T, alpha, tau, dt):
    # define the observed insolation based on observations from the IPCC AR6 Figure 7.2
    Q = 340  # W m^-2

    # find the new temperature using forward Euler method
    T_new = T + dt / C * (ASR(alpha, Q) - OLR(tau, T))

    return T_new

## Numerical Model

We can now use a loop to apply this function many times over by specifying an initial temperature and a time interval.

In [ ]:
# define the number of timesteps (currently years) to run the model
numtsteps = 15

# for converting number of seconds in a year
sec_2_yr = 3.154e7

# set the intial temperature (initial condition)
T_series = [288]

# set the initial time to 0
t_series = [0]

## Numerical Model

In [ ]:
# run the model
for n in range(numtsteps):
    # calculate and append the time since running the model, dependent on dt and the numtsteps
    t_series.append((n + 1) * dt / sec_2_yr)

    # calculate and append the new temperature using our pre-defined function
    T_series.append(step_forward(T_series[n], alpha=alpha, tau=tau, dt=dt))

# display the temeprature time series
print(T_series)

## Numerical Model

In [ ]:
# display the time series
print(t_series)

## Numerical Model

In [ ]:
# plot the results
fig, ax = plt.subplots()
ax.plot(t_series, T_series)
ax.set_xlabel("Time (years)")
ax.set_ylabel("Global mean temperature (K)")

# Revisiting the Climate Change Scenarios

## Enhanced Greenhouse Effect

We looked at how changing the transmissivity $\left(\tau\right)$ affected the equilibrium temperature $T_{\text{eq}}$.

Now we can use our time-dependent model to investigate this in more depth.

Reuse the model, this time setting $\tau = 0.57$.

In [ ]:
# define transmissivity (calculated previously from observations)
tau_2 = 0.57  # unitless number between 0 and 1

# define the number of timesteps (currently years) to run the model
numtsteps = 15

# set the intial temperature (initial condition)
T_series = [288]

# set the initial time to 0
t_series = [0]

# run the model
for n in range(numtsteps):
    # calculate and append the time since running the model, dependent on dt and the numtsteps
    t_series.append((n + 1) * dt / sec_2_yr)

    # calculate and append the new temperature using our pre-defined function
    T_series.append(step_forward(T_series[n], alpha=alpha, tau=tau_2, dt=dt))

## Enhanced Greenhouse Effect

In [ ]:
fig, ax = plt.subplots()
ax.plot(t_series, T_series)

ax.set_xlabel("Time (years)")
ax.set_ylabel("Global mean temperature (K)")

# Climate Feedbacks

## Ice-Albedo Feedback: Temperature Dependent Albedo

Our current model only contains one feedback, the 'Planck feedback' also called the 'Planck temperature response'. 

This feedback encapsulates that a warming of Earth leads to the planet emitting more energy. 

In reality, there are many  [climate feedbacks](https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_AnnexVII.pdf) that contribute to the Earth's net temperature change due to an energy imbalance. 

In this section, we will focus on incorporating an **ice-albedo feedback** into our model.

## Ice-Albedo Feedback: Temperature Dependent Albedo

When the Earth's surface warms, snow and ice melt.

This lowers the **albedo (**$\mathbf{\alpha}$**)**, because less solar radiation is reflected off Earth's surface. 

This lower albedo causes the climate to warm even more than if the albedo had stayed the same, increasing the snow and ice melt. This is referred to as a **positive feedback**.

Positive feedbacks amplify the changes that are already occurring. 

This particular feedback is referred to as the **ice-albedo feedback**.

## Ice-Albedo Feedback: Temperature Dependent Albedo

A simple way to parameterize ice-albedo feedback in our model is through a temperature-dependent albedo, such as the one defined below:

\begin{align}
\alpha = \left\{
        \begin{array}{cl}
        0.1 & T \gt 300 \text{ K} \\
        0.1 + (0.7-0.1) \cdot \frac{(T-300)^2}{(240-300)^2} & 240\text{ K} \le T \le 300\text{ K} \\
        0.7 & T \lt 240\text{ K}
        \end{array}
        \right.
\end{align}

Using this new temperature-dependent albedo, we can plot the graphs of absorbed shortwave radiation $\left(ASR\right)$ and outgoing longwave radiation $\left(OLR\right)$

## Ice-Albedo Feedback: Temperature Dependent Albedo

In [ ]:
# create a array ot temperatures to evaluates the ASR and OLR at
T = np.arange(200, 360, 2, dtype=np.float64)

# create empty arrays to fill with values later
ASR_vals = np.zeros_like(T)

# define the slope of the ramp function
m = (0.7 - 0.3) / (280 - 250)

# define the observed insolation based on observations from the IPCC AR6 Figure 7.2
Q = 340  # W m^-2

# define transmissivity
tau = 0.6127  # unitless number between 0 and 1

## Ice-Albedo Feedback: Temperature Dependent Albedo

In [ ]:
# define a function for absorbed shortwave radiation (ASR)
def ASR(Q, T):
    # define function for albedo
    if T >= 300:  # temperature of very warm and ice free earth.
        alpha = 0.1  # average albedo of land and sea without ice
    elif T > 240:  # temperature of Earth to sustain permafrost and sea ice everywhere.
        alpha = 0.1 + (0.7 - 0.1) * (T - 300) ** 2 / (240 - 300) ** 2
    else:
        alpha = 0.7  # average albedo of land and sea ice
    return (1 - alpha) * Q

# define a function for outgoing longwave raditation (OLR)
def OLR(tau, T):
    # define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
    sigma = 5.67e-8  # W m^-2 K^-4

    return tau * sigma * T**4


# calculate OLR for different values of T
OLR_vals = OLR(tau, T)

# calculate ASR for different values of T
for tt, temp in enumerate(T):
    ASR_vals[tt] = ASR(Q, temp)

## Ice-Albedo Feedback: Temperature Dependent Albedo

In [ ]:
# make plots
fig, ax = plt.subplots()
ax.plot(T, ASR_vals, label="Absorbed Shortwave Radiation ($ASR$)")
ax.plot(T, OLR_vals, label="Outgoing Longwave Radiation ($OLR$)")


ax.set_xlabel("Temperature (K)")
ax.set_ylabel("Radiative Flux (Wm$^{-2}$)")
ax.legend()

## Multiple Equilibria From Graphs

**Equilibrium temperatures** are solutions to the model equation when the rate of change of temperature is zero. There are two types of equilibrium solutions: *stable* and *unstable*.

  - A *stable equilibrium* temperature is a solution that the model asymptotes to (moves towards) over time. 
  - An *unstable equilibrium* temperature is a solution that the model diverges (moves away) from over time. The only time the model will stay at this equilibrium is if it starts *exactly* at the unstable equilibrium temperature. 
 
We can now incorporate the temperature-dependent albedo we defined above into our time-dependent model.

## Multiple Equilibria From Graphs

In [ ]:
# create a function to find the new temperature based on the previous using Euler's method.
def step_forward(T, tau, Q, dt):

    # define the heat capacity
    C = 286471954.64  # J m^-2K

    T_new = T + dt / C * (ASR(Q, T) - OLR(tau, T))

    return T_new

## Multiple Equilibria From Graphs

Let us explore how our model behaves under a variety of initial temperatures. We can use a `for` loop to compare different initial temperatures.

In [ ]:
dt = 60.0 * 60.0 * 24.0 * 365.0  # time interval, one year expressed in seconds

fig, ax = plt.subplots()
for init_temp in T:  # suite of initial temperatures in K
    numtsteps = 40  #  number of years to run the model

    # for converting the number of seconds in a year
    sec_2_yr = 3.154e7

    # set the initial temperature (initial condition)
    T_series = [init_temp]

    # set the initial time to 0
    t_series = [0]

    # run the model
    for n in range(numtsteps):

        # calculate and append the time since running the model, dependent on dt and the numtsteps
        t_series.append((n + 1) * dt / sec_2_yr)

        # calculate and append the new temperature using our pre-defined function
        T_series.append(step_forward(T_series[n], tau=tau, Q=Q, dt=dt))

    # make plot
    ax.plot(t_series, T_series)

ax.set_title("Temporal Evolution of Temperature")
ax.set_xlabel("Time (years)")
ax.set_ylabel("Temperature (K)")

## Finding Equilibria Numerically & Determining Convergence or Divergence

To verify the equilibrium solutions we identified graphically in the previous section, we can use python to find the exact values (i.e., where the rate of change in temperature is zero). 

That is find the temperatures that satisfy 

\begin{align}
0 = ASR-OLR.
\end{align}

To aid us, we will use [brentq](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html#scipy-optimize-brentq), a *root-finding function* from the `scipy` package.

## Finding Equilibria Numerically & Determining Convergence or Divergence

In [ ]:
# create function to find the forcing at the top of the atmosphere
def Ftoa(T):
    return ASR(Q, T) - OLR(tau, T)


#  brentq() requires a function and two end-points to be input as arguments
#  it will look for a root/zero of the function between those end-points
Teq1 = brentq(
    Ftoa, 200.0, 240.0
)  # these ranges are from the intersections of the graphs of ASR and OLR
Teq2 = brentq(Ftoa, 240.0, 280.0)
Teq3 = brentq(Ftoa, 280.0, 320.0)

print(Teq1, Teq2, Teq3)

## Finding Equilibria Numerically & Determining Convergence or Divergence

To assess the stability of these equilibria, we can plot the difference in $ASR$ and $OSR$. 

This is the same function `Ftoa` that we calculated in the previous cell, but we will recalculate it below for plotting purposes.

(get off slide show... too big :-)

In [ ]:
# we've already calculated ASR and OLR above
fig, ax = plt.subplots()
F = ASR_vals - OLR_vals
ax.plot(T, F, color="k", linewidth=3)

# find positive values and fill with red
pos_ind1 = T <= Teq1
ax.fill_between(T[pos_ind1], 0, F[pos_ind1], color="red")

pos_ind2 = (T >= Teq2) & (T <= Teq3)
ax.fill_between(T[pos_ind2], 0, F[pos_ind2], color="red")

# find negative values and fill with blue
neg_ind1 = (T >= Teq1) & (T <= Teq2)
ax.fill_between(T[neg_ind1], 0, F[neg_ind1], color="blue")

neg_ind2 = T >= Teq3
ax.fill_between(T[neg_ind2], 0, F[neg_ind2], color="blue")

# plot vertical lines/names at equilibrium temperatures
ax.axvline(x=Teq1, color="k", ls=":")
ax.axvline(x=Teq2, color="k", ls=":")
ax.axvline(x=Teq3, color="k", ls=":")

ax.annotate(
    "$T_{eq1}$",
    xy=(Teq1 - 5, -295),
    xytext=(Teq1 - 5, -295),
    rotation=90,
    annotation_clip=False,
)
ax.annotate(
    "$T_{eq2}$",
    xy=(Teq2 - 5, -295),
    xytext=(Teq2 - 5, -295),
    rotation=90,
    annotation_clip=False,
)
ax.annotate(
    "$T_{eq3}$",
    xy=(Teq3 - 5, -295),
    xytext=(Teq3 - 5, -295),
    rotation=90,
    annotation_clip=False,
)

# plot arrows/text to show stability of equilibrium points
ax.annotate(
    "",
    xy=(232, -50),
    xytext=(200, -50),
    arrowprops=dict(facecolor="black", arrowstyle="-|>"),
)

ax.annotate(
    "",
    xy=(242.5, -50),
    xytext=(233, -50),
    arrowprops=dict(facecolor="black", arrowstyle="<|-"),
)

ax.annotate(
    "",
    xy=(305.5, -50),
    xytext=(243.5, -50),
    arrowprops=dict(facecolor="black", arrowstyle="-|>"),
)

ax.annotate(
    "",
    xy=(358, -50),
    xytext=(307, -50),
    arrowprops=dict(facecolor="black", arrowstyle="<|-"),
)


ax.annotate("convergence", xy=(358, -170), xytext=(307, -170), rotation=90)

ax.annotate("divergence", xy=(305.5, -170), xytext=(243.5, -170), rotation=90)

ax.annotate("convergence", xy=(242.5, -170), xytext=(233, -170), rotation=90)


ax.set_xlabel("Temperature (K)")
ax.set_ylabel("$ASR-OLR$ (Wm$^{-2}$)");

## Finding Equilibria Numerically & Determining Convergence or Divergence

The red regions represent conditions where the Earth would warm, because the energy absorbed by the Earth system is greater than the energy emitted or reflected back into space. 

The blue regions represent conditions where the Earth would cool, because the outgoing radiation is larger than the absorbed radiation.

For example, if Earth started at an initial temperature below $T_{\text{eq1}}$ (in the left red region), it will move to the right on the $x$-axis, towards the $T_{\text{eq1}}$ equilibrium state.

Conversely, if Earth started between $T_{\text{eq1}}$ and $T_{\text{eq1}}$ (the left blue region), the temperature would decrease, moving left on the $x$-axis until it reaches $T_{\text{eq1}}$.

Thus $T_{\text{eq1}}$ is a *stable* equilibrium as the temperature curves will tend to this point after a long time.

## Changing Insolation: Effect on the Number Equilibrium Solutions

We learned that insolation (the amount of radiation Earth receives from the sun at the top of the atmosphere) fluctuates with time. 

Over Earth's history, the insolation has sometimes been lower, and sometimes higher, than the currently observed $340 \text{ Wm}^{-2}$.

These insolation changes directly affect the $ASR$, causing Earth to warm or cool depending on whether it receives more or less insolation respectively. 

To look at the effect that changing insolation has on Earth's equilibrium state(s), we can re-plot $ASR$ as a function of temperature for several different insolation values (including the temperature-dependent albedo), alongside the $OLR$.

## Changing Insolation: Effect on the Number Equilibrium Solutions

As we increase or decrease the insolation, the number of intersections between $ASR$ and $OLR$ can change! This means the number of equilibrium solutions for our model will also change.

In [ ]:
# define the observed insolation
Q_vals = [220, 340, 420]  # W m^-2

fig, ax = plt.subplots()
for Q_2 in Q_vals:
    # calculate ASR and OLR for different values of T
    for tt, temp in enumerate(T):

        ASR_vals[tt] = ASR(Q_2, temp)

    # make plots
    ax.plot(T, ASR_vals, label="$ASR$ for $Q$ = " + str(Q_2) + " Wm$^{-2}$")

# note we calculated OLR previously, and it does not depend on Q
ax.plot(T, OLR_vals, label="$OLR$")


ax.set_xlabel("Temperature (K)")
ax.set_ylabel("Radiative Flux (Wm$^{-2}$)")
ax.legend()

## Effect on Equilibrium Temperatures

To understand how this effect translates to different equilibrium temperatures of our model over time, we will apply a range of insolation values to our model.

Let us first start off with a very cold Earth, at $220 \text{K}$, and warm the Earth by steadily increasing the insolation above our present-day $340 \text{ Wm}^{-2}$ value.

## Effect on Equilibrium Temperatures

In [ ]:
# these are the values of insolation we will use
insolation_vals = np.arange(340, 500, 3)

# initial temperature we will use
init_temp = 220  # K

fig, ax = plt.subplots()

for i, insolation in enumerate(insolation_vals):  # suite of initial temperatures in K

    numtsteps = 100  #  number of years to run the model

    # for converting the number of seconds in a year
    sec_2_yr = 3.154e7

    # set the initial temperature (initial condition)
    T_series = [init_temp]

    # set the initial time to 0
    t_series = [0]

    # run the model
    for n in range(numtsteps):

        # calculate and append the time since running the model, dependent on dt and the numtsteps
        t_series.append((n + 1) * dt / sec_2_yr)

        # calculate and append the new temperature using our pre-defined function
        T_series.append(step_forward(T_series[n], tau=tau, Q=insolation, dt=dt))

    # make plot
    colors = plt.cm.coolwarm(np.linspace(0, 1, insolation_vals.shape[0]))
    if (
        insolation == 385
    ):  # This is just to highlight a particularly interesting insolation value
        ax.plot(t_series, T_series, color=colors[i], linestyle="dashed")
    else:
        ax.plot(t_series, T_series, color=colors[i])

ax.set_ylabel("Temperature (K)")
ax.set_xlabel("Time (years)")

## Questions?

## See you in the next lecture!

## Equilibrium Climate Sensitivity (Your turn)

Here we define the **[equilibrium climate sensitivity](https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_AnnexVII.pdf)** as the long-term global warming (equilibrium temperature increase) caused by a doubling of carbon dioxide above its pre-industrial concentration. 

The impact of a doubling of carbon dioxide on these energy flows is measured by a [radiative forcing](https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_AnnexVII.pdf). 

Here a positive radiation forcing leads to warming, and a negative radiative forcing leads to cooling.

The equilibrium climate sensitivity depends on a number of things, including physics and feedbacks of the model used. In the following exercise, you will calculate the equilibrium climate sensitivity of _our_ model.

1. Write a function called `step_forward()` as from above, and then create another function that adds in a radiative forcing to the difference between $ASR$ and $OLR$ and call it `step_forward_forced()`. 

Make sure both functions output the energy balance of the model. 

Consistent with the [IPCC AR6](https://www.ipcc.ch/report/ar6/wg1/chapter/chapter-7/#Effective), use an effective radiative forcing of 3.93 W m$^{-2}$, where _effective_ means the climate system, but not the surface temperature, has been allowed to adjust.

## Equilibrium Climate Sensitivity (Your turn)

In [ ]:
# define your functions and constants

# define albedo
alpha = 0.2941  # unitless number between 0 and 1

# define transmissivity
tau = 0.6127    # unitless number between 0 and 1

# effective radiative forcing for a doubling of CO2
F = 3.93  # W/m^2

# define the time interval, one year expressed in seconds
dt = 60.0 * 60.0 * 24.0 * 365.0

# for converting number of seconds in a year
sec_2_yr = 3.154e7


# define a function for absorbed shortwave radiation (ASR)
def ASR(alpha, Q):
    return (1 - alpha) * Q


# define a function for outgoing longwave radiation (OLR)
def OLR(tau, T):
    # define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
    sigma = 5.67e-8  # W m^-2 K^-4

    return tau * sigma * T**4


# create a function to find the new temperature based on the previous using Euler's method.
def step_forward(T, alpha, tau, dt):
    # define the observed insolation based on observations from the IPCC AR6 Figure 7.2
    ...

    # define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
    ...

    Ftoa = ...

    T_new = ...

    return ...


# create a function to find the new temperature based on the previous using Euler's method.
def step_forward_forced(T, alpha, tau, dt):
    # define the observed insolation based on observations from the IPCC AR6 Figure 7.2
    ...

    # define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
    ...

    Ftoa = ...

    T_new = ...

    return ...

## Equilibrium Climate Sensitivity (Your turn)

2. Using an initial temperature of 288 K, run the model to equilibrium. Make sure your model is in equilbrium by checking that the energy balance is near zero.

In [ ]:
# run the model to equilibrium without forcing and begining with T(0) = 288K

# define the number of timesteps (years) to run the model
numtsteps = 40

# set the intial temperature (initial condition)
T_series = [288]

# set the initial time to 0
t_series = [0]

# run the model
for n in range(numtsteps):
    # calculate and append the time since running the model, dependent on dt and the numtsteps
    t_series.append(...)

    # calculate and append the new temperature using our pre-defined function and get energy balance
    ...
    T_series.append(...)

print(...)

## Equilibrium Climate Sensitivity (Your turn)

3. Run the forced model equilibrium using the unforced equilibrium temperature as your inital condition and the `step_forward_forced()` function you wrote above.


In [ ]:
# define the number of timesteps (years) to run the model
numtsteps = 40

# set initial condition (temperature) to the equilibrium value from the last run without forcing
T_series_forced = [T_series[-1]]

# set the initial time to 0
t_series_forced = [0]

# run the model
for n in range(numtsteps):
    # calculate and append the time since running the model, dependent on dt and the numtsteps
    t_series_forced.append(...)

    # calculate and append the new temperature using our pre-defined function and get energy balance
    ...
    T_series_forced.append(...)

print(...)

## Equilibrium Climate Sensitivity (Your turn)

4. Plot the temperature curve from the forced simulation as a function of time.

In [ ]:
# plot the time series
fig, ax = plt.subplots()
_ = ...

ax.set_xlabel("Time (years)")
ax.set_ylabel("Global mean temperature (K)")

## Equilibrium Climate Sensitivity (Your turn)

5. Subtract the intial temperature used for your forced simulation from the final temperature after running to equilibrium to get the equilibrium climate sensitivty.


In [ ]:
# calculate equilibrium climate sensitivity
ecs = ...
print("Equilibrium Climate Sensitivity: ", ecs)